In [7]:
import fastText
import spacy
import torch 
import torchtext
import torchtext.data as data
import pandas as pd
import string

In [12]:
vec = torchtext.vocab.Vectors('wiki-news-300d-1M.vec', max_vectors=100000)

In [33]:
vecHindi = torchtext.vocab.Vectors('wiki.hi.vec', max_vectors=100000)

In [21]:
vecHindi.itos[:20]

['</s>', 'के', ',', 'में', 'की']

In [5]:
vec.vectors[vec.stoi['the']].shape

torch.Size([300])

In [16]:
def getVector(vec, word):
    return vec.vectors[vec.stoi[word]]

def getDistance(word1, word2):
    dist = torch.dot(word1, word2)
    return dist

def getDiff (word1, word2):
    dif = word1 - word2
    return torch.sum(dif)

def getBiasOcc(vec, word1, word2, vec3):
    vec1 = getVector(vec, word1)
    vec2 = getVector(vec, word2)
    dis = getDistance(vec1, vec3)
    dis2 = getDistance(vec2, vec3)
    return dis-dis2
    
def getBias(vec, word1, word2, Tword):
    vec1 = getVector(vec, word1)
    vec2 = getVector(vec, word2)
    vec3 = getVector(vec, Tword)
    dis = getDistance(vec1, vec3)
    dis2 = getDistance(vec2, vec3)
    return dis-dis2

In [45]:
b1 = getBias(vec, "woman", "man", "engineer")
b2 = getBias(vecHindi, "स्त्री", "मर्द", "इंजीनियर")
print ("English bias::: ", b1)
print("Hindi bias::: ", b2)

b1 = getBias(vec, "woman", "man", "intelligent")
b2 = getBias(vecHindi, "स्त्री", "मर्द", "बुद्धिमान")
print ("English bias::: ", b1)
print("Hindi bias::: ", b2)

b1 = getBias(vec, "woman", "man", "children")
b2 = getBias(vecHindi, "स्त्री", "मर्द", "बच्चे")
print ("English bias::: ", b1)
print("Hindi bias::: ", b2)

English bias:::  tensor(-0.0163)
Hindi bias:::  tensor(-0.6024)
English bias:::  tensor(-0.1282)
Hindi bias:::  tensor(-2.6024)
English bias:::  tensor(0.4104)
Hindi bias:::  tensor(1.0463)
tensor(-0.0184)


In [4]:
dfs = pd.read_csv("employmentData.csv")


In [71]:
def getBiasLists(vec, dfs, JobName):
    wordList = []
    occBiasDict = {}
    malePronoun = ['he', 'him', 'man']
    femalePronoun = ['she', 'her', 'woman']
    badList = ["and", "other", 'all', "Miscellaneous"]
    ordListJob = []
    orderListCens =[]
    numPronouns = 3.0
    for j, job in enumerate(dfs['Unnamed: 0']):
        noUse = False
        if not isinstance(job, str):
            continue
        for i, word in enumerate(job.split()):
            word = word.strip()
            translator=str.maketrans('','',string.punctuation)
            string_name=word.translate(translator)
            if string_name in badList:
                continue
            try:
                emb = getVector(vec, string_name)
            except:
                wordList.append(string_name)
                noUse = True
                break
            if i == 0:
                occVec = emb
            else:
                occVec += emb
        if not noUse:
            if dfs['Unnamed: 2'][j] is not "-":
                censusNum = float(dfs['Unnamed: 2'][j])
            else:
                censusNum = 0
            bias= 0
            for woman, man in zip(femalePronoun, malePronoun):
                bias += getBiasOcc(vec, woman, man, occVec)
            bias = bias/numPronouns
            occBiasDict[job] = (bias, censusNum)
            ordListJob.append((bias,censusNum,job))
            orderListCens.append((censusNum,bias,job))
        ordListJob = sorted(ordListJob)
        orderListCens = sorted(orderListCens)
    return occBiasDict, ordListJob, orderListCens

In [72]:
occBiasDict, ordListJob, orderListCens = getBiasLists(vec, dfs)

In [73]:
for key in ordListJob:
    print (key[2],occBiasDict[key[2]])

Computer control programmers and operators (tensor(-24.6564), 9.7)
Computer hardware engineers (tensor(-24.5842), 18.9)
Computer operators (tensor(-24.5746), 63.8)
Computer network architects (tensor(-24.3837), 10.3)
Computer occupations, all other (tensor(-24.3200), 21.9)
Computer support specialists (tensor(-24.2580), 28.1)
Computer programmers (tensor(-24.1488), 21.2)
Computer systems analysts (tensor(-24.0609), 37.5)
Computer and information research scientists (tensor(-23.8184), 0)
Computer and information systems managers (tensor(-23.8061), 27.0)
Computer and mathematical occupations (tensor(-23.7147), 25.6)
Electrical, electronics, and electromechanical assemblers (tensor(-15.8637), 49.2)
Electrical and electronics engineers (tensor(-15.4708), 9.4)
Aircraft pilots and flight engineers (tensor(-13.6275), 9.0)
Financial clerks, all other (tensor(-13.5099), 61.9)
Financial specialists, all other (tensor(-13.4132), 0)
Aircraft structure, surfaces, rigging, and systems assemblers (te

In [65]:
for key in orderListCens:
    print (key[2],occBiasDict[key[2]])

Computer and information research scientists (tensor(-22.6954), 0)
Financial specialists, all other (tensor(-12.8271), 0)
Financial examiners (tensor(-12.7167), 0)
Aircraft structure, surfaces, rigging, and systems assemblers (tensor(-12.6656), 0)
Mining and geological engineers, including mining safety engineers (tensor(-12.1739), 0)
Agricultural inspectors (tensor(-9.8657), 0)
Agricultural and food science technicians (tensor(-9.7365), 0)
Agricultural and food scientists (tensor(-9.7273), 0)
Agricultural engineers (tensor(-9.6328), 0)
Ship engineers (tensor(-9.1724), 0)
Ship and boat captains and operators (tensor(-8.9242), 0)
Conservation scientists and foresters (tensor(-8.5175), 0)
Environmental engineers (tensor(-7.8983), 0)
Compensation, benefits, and job analysis specialists (tensor(-7.5451), 0)
Railroad brake, signal, and switch operators (tensor(-7.4839), 0)
Compensation and benefits managers (tensor(-7.3006), 0)
Marine engineers and naval architects (tensor(-7.1972), 0)
Nucl

In [74]:
print(len(orderListCens))

405


In [76]:
dfsH = pd.read_csv("employmentIndia.csv")


In [93]:
def getBiasLists(vec,vecH, dfs):
    wordList = []
    occBiasDict = {}
    malePronoun = ['उसे', "मर्द", 'आदमी']
    femalePronoun = ['उसकी', "स्त्री", 'महिला']
    badList = ["and", "other", 'all', "Miscellaneous"]
    ordListJob = []
    orderListCens =[]
    numPronouns = 3.0
    for j, job in enumerate(dfs['Unnamed: 0']):
        noUse = False
        if not isinstance(job, str):
            continue
        for i, word in enumerate(job.split()):
            word = word.strip()
            translator=str.maketrans('','',string.punctuation)
            string_name=word.translate(translator)
            if string_name in badList:
                continue
            try:
                emb = getVector(vec, string_name)
            except:
                wordList.append(string_name)
                noUse = True
                break
            if i == 0:
                occVec = emb
            else:
                occVec += emb
        if not noUse:
            if dfs['Perc'][j] is not "-" or dfs['Perc'][j] is not None :
                censusNum = float(dfs['Perc'][j])
            else:
                censusNum = 0
            bias= 0
            for woman, man in zip(femalePronoun, malePronoun):
                bias += getBiasOcc(vecH, woman, man, occVec)
            bias = bias/numPronouns
            occBiasDict[job] = (bias, censusNum)
            ordListJob.append((bias,censusNum,job))
            orderListCens.append((censusNum,bias,job))
        ordListJob = sorted(ordListJob)
        orderListCens = sorted(orderListCens)
    return occBiasDict, ordListJob, orderListCens

In [94]:
occBiasDictH, ordListJobH, orderListCensH = getBiasLists(vec, vecHindi, dfsH)

In [95]:
for key in ordListJobH:
    print (key[2],occBiasDictH[key[2]])

Shoe Cleaning and Other Street Services Elementary Occupations (tensor(-2507.6023), 0.07435719249478799)
Fashion and Other Models (tensor(-2040.8380), 0.25531914893617)
Power Production Plant Operators (tensor(-1516.4650), 0.09507149376330999)
Fashion and Other Models (tensor(-2040.8380), 0.25531914893617)
Shoe Making and Related Machine Operators (tensor(-1163.4941), 0.11906485983241999)
Agricultural and Other Mobile Plant Operators (tensor(-1159.5922), 0.0531613240558259)
Helpers and Cleaners in Offices, Hotels and Other Establishments (tensor(-1130.2701), 0.17316339735413802)
Hand Pedal Vehicle Drivers (tensor(-1124.3859), 0.018968964333771)
Pelt, Leather and Shoe Making Trades Workers (tensor(-1064.4658), 0.250949964452943)
Shoe Makers and Related Workers (tensor(-1044.6041), 0.248457945834071)
Power Production and Related Plant Operators (tensor(-997.7166), 0.0943586299529886)
Cement and Other Mineral Products Machine Operators (tensor(-960.9315), 0.134533183352081)
Brewers, Wine 